## Usage of sclang in sc3nb

You can send commands and receive data directly from the SuperCollider Language

In [ ]:
import time
import numpy as np

import sc3nb as scn

In [ ]:
sc = scn.startup()

### sclang command execution

To send sc3 commands (i.e. program strings) to the language, either use the functions
- **`cmd()`** normal command sending.
- **`cmdv()`** verbose, i.e. returning string is collected and output to notebook ( alias for `cmd(.., verbose=True)` )
- **`cmdg()`** send program and get and parse the output ( alias for `cmd(.., get_return=True)` )

In [ ]:
# sc.cmd(cmdstr, pyvars)
sc.lang.cmd('"hello".postln')  # check jupyter console for output

use cmdv If sclang output should be displayed as cell output

In [ ]:
sc.lang.cmdv('"sc3nb".postln')  # also check jupyter console for output

or use the corresponding Magics in Jupyter

* Jupyter line magics **%sc, %scv, %scg, %scgv, %scgs, %scs**
* Jupyter cell magics **%%sc, %%scv, %%scg, %%scgv, %%scgs, %%scs**

which wrap the above functions: (v=verbose, g=get, s=silent, verbose is default, so %sc=%scv)
See examples below

Cell magics can be placed within code just as the function calls as shown here:

In [ ]:
%sc x = Synth.new(\default, [\freq, 100])
for p in range(1, 10):  # a bouncing ball
    %scs Synth.new(\s1, [\freq, 200])  // this is sc cell so use sc3 comments instead of # 
    time.sleep(1/p)
# NOTE: Windows needs delay between sclang inputs, so use sleep
%sc x.free 

Use raw python strings for multi-line sc3-programs:

In [ ]:
sc.lang.cmd(r"""
Routine({
    x = 10.collect{ |i|
        0.2.wait;
        Synth.new(\default, [\freq, 50+(50*i)]);
    };
    1.wait;
    x.do{|e| 
        e.release;
        0.1.wait;};
}).play;
""")

alternatively, you can use the cell magic:

In [ ]:
%%sc
Routine({
    x = 5.collect{ |i|
        0.2.wait;
        Synth.new(\default, [\freq, 50+(50*i)]);
    };
    1.wait;
    x.do{|e| 
        e.release;
        0.1.wait;};
}).play;

Note that the code is executed in sclang and python is returning directly after sending the command.

### sclang command execution with python variable injection

Python variables can be injected into sc3 commands by using the ^ special: The following examples demonstrates it by setting frequencies by using python variables

In [ ]:
for p in range(1, 50):  # a tone ladder
    freq = 50 + p*3
    dur = np.log(p)
    position = np.sign(p-25)
    %scs Synth.new(\s1, [\freq, ^freq, \dur, ^dur, \pan, ^position])
    time.sleep(0.05)

This is injection is done with

In [ ]:
scn.sclang.convert_to_sc?

In [ ]:
python_list = [1,2,3,4]
%sc ^python_list.class

In [ ]:
complex_py = 1+1j
%sc ^complex_py.class

In [ ]:
symbol = r"\\python"
%sc ^symbol.class

When using the `cmd` | `cmdv` | `cmdg` `cmds` functions you can also provide a dictionary with variable names as keys and content as values (which can use other python vars or statements)

In [ ]:
sc.lang.cmdv("^name1 / ^name2", pyvars={'name1': 9,'name2': 9*2})

In [ ]:
# without providing pyvars, variables are searched in the users namespace.
freq = 5
rate = 6
sc.lang.cmdv("(^freq + 1) * (^rate + 1)")

In [ ]:
# alternatively via the magic this is done as:
%scv (^freq + 1) * (^rate + 1)

### Getting sclang output in python

* To *get* the output of an sclang snippet into a python variable, use the cmdg function.
* The following example shows how to transfer a synth's nodeID

In [ ]:
# start a Synth
sc.lang.cmd(r"""x = Synth.new(\default)""")

In [ ]:
# get the nodeId to python
nodeID = sc.lang.cmdg("x.nodeID")
print(nodeID)

In [ ]:
# use the nodeID to free the Synth via a message to scsynth audio server directly
sc.server.msg("/n_free", nodeID)  

**sc.cmdg(), resp. %scg return integers, floats, strings and lists**
* %scg can be assigned to a python variable within code

In [ ]:
a = %scg 1234 + 23452
print(f"returned an {type(a)} of value {a}")

In [ ]:
a = %scg 1234.5.squared
print(f"returned an {type(a)} of value {a}")

In [ ]:
a = %scg "sonification".scramble
print(f"returned an {type(a)} of value {a}")

Be careful when using magics directly after another!

In [ ]:
%sc ~retval = "sonification".scramble
time.sleep(0.001)  # (in Windows) without pause an Empty might be thrown..
%scg ~retval

(in Windows) it should be prefered to combine your code in a single `cmd()` execution.

In [ ]:
scramble = %scg ~retval = "sonification".scramble; ~retval ++ "!";
scramble

In [ ]:
a = %scg (1,1.1..2)

In [ ]:
a

In [ ]:
a = %scg (1,1.1..2)
print(f"list with length: {len(a)}")
a

Note that floating-point numbers do only have limited precision

In [ ]:
[round(num, 6) for num in a]

However they should be close

In [ ]:
np.allclose(a, np.arange(1, 2, 0.1))

### Cell magics

In [ ]:
%sc {SinOsc.ar(MouseX.kr(200,400))}.play  // move mouse horizontally, CMD-. to stop

In [ ]:
%sc s.scope()

In [ ]:
%sc s.freeAll

In [ ]:
value = %scg 75-33
print("value = ", value)

In [ ]:
%%sc
{
    x = Synth.new(\s2, [\freq, 100, \num, 1]);
    250.do{|i|
        x.set(\freq, sin(0.2*i.pow(1.5))*100 + 200);
        0.02.wait;
    };
    x.free;
}.fork

* Try %scv and %%scv for verbose lines resp. cells
* Try %scg and %%scg for getter lines resp. cells
* Try %scs and %%scs for silent lines resp. cells

### Stop synths

In [ ]:
synth = "default"
%sc x = Synth.new(^synth)

* to stop all playing synths either use CMD-. (in Jupyter Command mode).
* It is a shortcut for the ´free_all´ method of the default server

In [ ]:
%sc s.freeAll  // leaves the s.scope running

In [ ]:
scn.SC.default.server.free_all()

which is also available using our `sc` instance directly

In [ ]:
sc.server.free_all()

In [ ]:
sc.exit()